In [152]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.mixture import GaussianMixture
import os
import pickle
from feature_extraction import *
import warnings
warnings.filterwarnings('ignore')

In [153]:
TRAIN_PATH = "../data/processed/train2/"
# TEST_PATH = "../data/processed/test/"
REF_PATH = "../data/processed/reference/"
WRITE_PATH = "../data/models/"

In [11]:
def setup_df(PATH):
    all_files = os.listdir(PATH)
    df = pd.DataFrame(all_files, columns=['file'])
    df['speaker'] = df['file'].apply(lambda x: int(x.split('.')[0].split('-')[-1]))
    # 0 for neutral, 1 for emotional
    df['speech_type'] = df['file'].apply(lambda x: 0 if int(x.split('.')[0].split('-')[2])==1 else 1)
    df['F0_contour'] = df['file'].apply(lambda x: get_F0_contour(PATH+x))
    return df

In [12]:
def get_audio_features(df, norm=False):
    if norm==False:
        df['SQ25'] = df['F0_contour'].apply(get_SQ25)
        df['SQ75'] = df['F0_contour'].apply(get_SQ75)
        df['IDR'] = df['F0_contour'].apply(get_IDR)
        df['voiced_segments'] = df['F0_contour'].apply(get_voiced_segments)
        df['F0_median'] = df['F0_contour'].apply(get_F0_median)
        df['sdmedian'] = df['F0_contour'].apply(get_sdmedian)
    else:
        df['SQ25'] = df['F0_contour_norm'].apply(get_SQ25)
        df['SQ75'] = df['F0_contour_norm'].apply(get_SQ75)
        df['IDR'] = df['F0_contour_norm'].apply(get_IDR)
        df['voiced_segments'] = df['F0_contour_norm'].apply(get_voiced_segments)
        df['F0_median'] = df['F0_contour_norm'].apply(get_F0_median)
        df['sdmedian'] = df['F0_contour_norm'].apply(get_sdmedian)
    
    df['SVMeanRange'] = df['voiced_segments'].apply(get_voiced_segment_range)
    df['SVMaxCurv'] = df['voiced_segments'].apply(get_max_voiced_curvature)
    
    return df

In [13]:
ref_df = setup_df(REF_PATH)
ref_df = get_audio_features(ref_df)
temp_df = setup_df(TRAIN_PATH)
# test_df = setup_df(TEST_PATH)

## Training the GMMs on the Reference Data

In [154]:
trained_GMMs = {}
features = [ 'SQ25', 'SQ75','F0_median', 'sdmedian', 'IDR', 'SVMeanRange','SVMaxCurv']
# features = [ 'SQ75','IDR', 'SVMeanRange','SVMaxCurv']

for feature in features:
    gmm = GaussianMixture(n_components=2)
    gmm.fit(ref_df[feature].values.reshape(-1,1))
    trained_GMMs[feature] = gmm

## IFN Algo

In [179]:
ref_df['F0_contour_sum'] = ref_df['F0_contour'].apply(sum)

In [180]:
ref_df['F0_contour_length'] = ref_df['F0_contour'].apply(len)

In [181]:
ref_df['F0_contour_mean'] = ref_df['F0_contour_sum']/ref_df['F0_contour_length']

In [182]:
avg_F0_ref = np.sum(ref_df['F0_contour_mean']) / 24

In [183]:
def stratified_sample_df(df):
    per_speaker_neutral_number = df[df['speech_type']==0]['speaker'].value_counts().values[0]
    sampled_df = []
    sampled_df.append(df[df['speech_type']==0])
    speakers = df['speaker'].unique()
    for speaker in speakers:
        speaker_df = df[df['speaker'] == speaker]
        speaker_emotional_df= speaker_df[speaker_df['speech_type']==1]
        sampled_df.append(speaker_emotional_df.sample(n=per_speaker_neutral_number))
    return pd.concat(sampled_df).reset_index(drop=True)

In [184]:
def infer_GMM(df):
    infered = np.zeros(shape=(1,len(features)))
    for i in range(len(features)):
        res = trained_GMMs[features[i]].score_samples([[df[features[i]]]])
        infered[:,i] = res
    return infered[0]

In [185]:
def get_changed_labels(neutral_list, emotional_list, row):
    if row['file'] in neutral_list:
        return 0
    else:
        return 1

In [186]:
def get_S_s_F0(F0_ref, df):
#     grouped_df_neu = df[df['speech_type']==0].groupby(['speaker']).sum().reset_index(level=0)
#     grouped_df_neu = grouped_df_neu[['speaker','F0_contour_sum','F0_contour_length']]
#     grouped_df_neu['F0_s_neu'] = grouped_df_neu['F0_contour_sum'] / grouped_df_neu['F0_contour_length']
#     grouped_df_neu['S_s_F0'] = F0_ref / grouped_df_neu['F0_s_neu']

    df_neu = df[df['changed_speech_type']==0]
    speakers = df['speaker'].unique()
    grouped_df_neu = {}
    UPPER_CAP = 100
    LOWER_CAP = 0.00001
    for speaker in speakers:
        speaker_df_neu = df_neu[df_neu['speaker']==speaker]
        speaker_mean = (speaker_df_neu['F0_contour_sum']/speaker_df_neu['F0_contour_length']).mean()
        if F0_ref/speaker_mean > UPPER_CAP:
            grouped_df_neu[speaker] = UPPER_CAP
        elif F0_ref/speaker_mean < LOWER_CAP:
            grouped_df_neu[speaker] = LOWER_CAP
        else:
            grouped_df_neu[speaker] = F0_ref/speaker_mean
    
    
        
        
    
#     return grouped_df_neu[['speaker','S_s_F0']].set_index('speaker').to_dict()['S_s_F0']
    return grouped_df_neu

def get_normalised_df(df, avg_F0_ref=avg_F0_ref, S0_func=get_S_s_F0):
    df_norm = S0_func(avg_F0_ref, df)
    df['F0_contour_norm'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)
#     df['F0_contour'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)

    return df_norm, df
        

def get_normalised_df_infer(df, df_norm):
    df['F0_contour_norm'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)
#     df['F0_contour'] = df.apply(lambda x: x['F0_contour']/df_norm[x['speaker']], axis=1)

    return df_norm, df

In [187]:
def get_pred_labels(df):
    
    grouped_sampled_df = df.groupby('speaker')
    neutral = []
    emotional = []

    for name, group in grouped_sampled_df:
        neu_result = group[group['predicted_likelihood'] >= 0.5]
        emo_result = group[group['predicted_likelihood'] < 0.5]

        total = group.shape[0]
        to_add = int(np.ceil(0.2 * total)) - neu_result.shape[0]
        converted_neu_add = neu_result
        if to_add > 0:
            emo_result_sort = emo_result.sort_values('predicted_likelihood', ascending=False)
            converted_neu_add = emo_result_sort.head(to_add)
            emo_result.drop(converted_neu_add.index, inplace=True)

        neutral.extend(converted_neu_add['file'].tolist())
        emotional.extend(emo_result['file'].tolist()) 

    df['changed_speech_type'] = df.apply(lambda x: \
                                         get_changed_labels(neutral,emotional,x), axis=1) 
    
    return df
    

In [188]:
def get_stopping_criteria(df, count):
    df = get_pred_labels(df)
    
    if count==0:
        return df, 1000000

    else:
        changed_dict = (df['prev_changed_speech_type'] != df['changed_speech_type']).\
                        value_counts().to_dict()

        if True not in changed_dict.keys():
            changed_dict[True] = 0
            epsilon = changed_dict[True]/changed_dict[False]
        elif False not in changed_dict.keys():
            epsilon = 1000000
        else:
            epsilon = changed_dict[True]/changed_dict[False]
    
    return df, epsilon

In [191]:
EXPERIMENT_TYPE = 'norm_original_ldc_0.5'

In [190]:
# for every epoch
epoch = 0

ITERATION_CLF_REPORT = []
ITERATION_FILE_CHANGE = []
LDC_CLFS = []


for iterations in range(400):

    sampled_df = stratified_sample_df(temp_df)
    
    train_df, test_df = train_test_split(sampled_df, test_size = 0.33, stratify = sampled_df[['speaker','speech_type']])
    
    train_df = get_audio_features(train_df)
    
    train_df['F0_contour_sum'] = train_df['F0_contour'].apply(sum)
    train_df['F0_contour_length'] = train_df['F0_contour'].apply(len)

    test_df['F0_contour_sum'] = test_df['F0_contour'].apply(sum)
    test_df['F0_contour_length'] = test_df['F0_contour'].apply(len)
    
    count = 0

    # for 400 iterations
    sampled_df_norm = None
    ldc_clf = None
    epsilon = 1000000
    max_iters = 1000
    
    ldc_clf = LinearDiscriminantAnalysis(solver='lsqr')
    
    print('*************************************************') 
    print('*************************************************') 

            
    for stage in ['train', 'test']:

        print('----------------------------------------')   
        print('Stage - ', stage)


        if stage == 'train':
            
            ITERATION_FILE_CHANGE_ELEM = []
            
            while epsilon > 0.05 and count < max_iters:
        
                print('=========================================')   
                
                print(sampled_df_norm)

                if count ==0 :
                    train_df['F0_contour_sum'] = train_df['F0_contour'].apply(sum)
                    train_df['F0_contour_length'] = train_df['F0_contour'].apply(len)
                    train_df = get_audio_features(train_df, norm=False)
                else:
                    train_df['F0_contour_sum'] = train_df['F0_contour_norm'].apply(sum)
                    train_df['F0_contour_length'] = train_df['F0_contour_norm'].apply(len)
                    # Change above to F0_contour_norm when norm=True
                    train_df = get_audio_features(train_df, norm=True)
                train_df['inferred'] = train_df.apply(infer_GMM,axis=1)
                
                ldc_clf.fit(np.array(train_df['inferred'].tolist()),train_df['speech_type'].values)

                train_df['predicted_likelihood'] = ldc_clf.predict_proba(np.array(train_df['inferred'].tolist()))[:,0]

                train_df, epsilon = get_stopping_criteria(train_df, count)
                
                print(count)
                print(epsilon)
                ITERATION_FILE_CHANGE_ELEM.append(epsilon)
                LDC_CLFS.append(ldc_clf)
                
                
                train_df['prev_changed_speech_type'] = train_df['changed_speech_type']
                                
                sampled_df_norm, train_df = get_normalised_df(train_df, avg_F0_ref=avg_F0_ref, S0_func=get_S_s_F0)                
                count+=1
                
            ITERATION_FILE_CHANGE.append(ITERATION_FILE_CHANGE_ELEM)

        else:
                sampled_df_test = test_df
                if count!=0:
                    _, sampled_df_test = get_normalised_df_infer(test_df, sampled_df_norm)
                if count == 0 :
                    sampled_df_test = get_audio_features(sampled_df_test, norm=False)
                else:
                    sampled_df_test = get_audio_features(sampled_df_test, norm=True)
                sampled_df_test['inferred'] = sampled_df_test.apply(infer_GMM,axis=1)

                sampled_df_test['predicted_likelihood'] = ldc_clf.predict_proba(np.array(sampled_df_test['inferred'].tolist()))[:,0]
#                 sampled_df_test['changed_speech_type'] = sampled_df_test['predicted_likelihood'].\
#                                                             apply(lambda x: 0 if x >=0.7 else 1)

                sampled_df_test = get_pred_labels(sampled_df_test)

                clf_report = classification_report(sampled_df_test['speech_type'],sampled_df_test['changed_speech_type'])
                ITERATION_CLF_REPORT.append(clf_report)
                print(clf_report)
                
                
                
#         count+=1



*************************************************
*************************************************
----------------------------------------
Stage -  train
None
0
1000000
{7: 0.8316650684456629, 5: 1.574040595980181, 18: 0.7414497205452484, 4: 0.7419840919250125, 20: 0.7588233594116848, 15: 1.2326961833367505, 17: 1.4581717550092794, 24: 0.8543772388301906, 3: 1.1741676588517116, 11: 1.3342313749085697, 19: 1.241052686930941, 21: 1.2969684999338218, 13: 1.4481174577755098, 23: 1.0516326692992857, 9: 1.3223657090599676, 22: 0.8314085468944288, 10: 0.7253889037105636, 8: 0.8088438298929645, 2: 0.7918143927849646, 12: 0.7446493586319737, 6: 0.6062088889945618, 14: 0.778223908684726, 16: 0.8259355424432687, 1: 1.3610288563356672}
1
0.12941176470588237
{7: 0.9836093141937676, 5: 1.8417109124630549, 18: 0.6229827998922347, 4: 0.5425461839844089, 20: 0.5758128907888348, 15: 1.519539880412992, 17: 2.126264867106842, 24: 0.7299604662311001, 3: 1.3786696910933092, 11: 1.7801733617904125, 19: 1.5

0
1000000
{16: 0.8370163430550822, 18: 0.7723130092815123, 11: 1.3145060209444734, 12: 0.7131760366865915, 9: 1.4502349447127156, 4: 0.7696832369281622, 5: 1.4342490396537675, 7: 0.8787247586421745, 17: 1.2936353094246633, 23: 1.2207577779023033, 13: 1.523491714976205, 8: 0.8156993618873859, 6: 0.6508223963735247, 24: 0.6994125642793864, 20: 0.7362803355263234, 21: 1.4171912045976771, 1: 1.5335000885116077, 15: 1.0789258280383183, 2: 0.7378016963270727, 19: 1.0798104499166747, 3: 0.8118272328552879, 22: 0.8087379383354043, 14: 0.7213237106297268, 10: 0.6422811267255104}
1
0.14285714285714285
{16: 0.7005963585413031, 18: 0.5964673843054654, 11: 1.727926079099272, 12: 0.5086200593039945, 9: 2.103181394865894, 4: 0.5924122852082134, 5: 1.7488797245419518, 7: 0.7093306226200508, 17: 1.673492313790244, 23: 1.4902495523089687, 13: 2.3210270056011386, 8: 0.6942079343453956, 6: 0.46711342114426213, 24: 0.48917793507186674, 20: 0.5648380878572842, 21: 2.008430910389015, 1: 2.2234559625468666, 1

0
1000000
{14: 0.822415994488209, 23: 1.2395688727848955, 20: 0.7989339986926389, 7: 1.0020287738382543, 4: 0.7696832369281622, 6: 0.643864638779979, 18: 0.8201481665382381, 10: 0.7006144217990822, 9: 1.708439173014334, 13: 1.2295929627121658, 3: 1.0272476958602805, 22: 0.7927423363014519, 2: 0.6897728941206341, 16: 0.8858682751869361, 8: 0.8747698785354947, 21: 1.7696090629032315, 11: 1.0490868169105945, 15: 1.165275882248662, 17: 1.4145241877245294, 24: 0.6556118157885273, 5: 1.8111165117151151, 12: 0.7150699523287385, 1: 1.4800817550263232, 19: 1.2689097933908624}
1
0.18518518518518517
{14: 0.6763680679900299, 23: 1.5365309903772155, 20: 0.6594942776117515, 7: 1.0040616635997957, 4: 0.5924122852082134, 6: 0.4126307110792319, 18: 0.6726430150760336, 10: 0.5586780119667977, 9: 2.9187644078899004, 13: 1.5118988539512812, 3: 0.8934830463686497, 22: 0.6284404117646843, 2: 0.5423183101559889, 16: 0.7847626009826774, 8: 0.7652223403930042, 21: 2.7078950004118787, 11: 1.1081882963764684, 15

1
0.21518987341772153
{1: 1.8207044408925772, 9: 1.168701702419197, 11: 1.8607282419054332, 4: 0.553334689894988, 7: 1.039043827362946, 3: 0.9911413866004614, 18: 0.6165902087907557, 10: 0.4957639119433038, 24: 0.5218220882887014, 19: 1.018726607083836, 12: 0.55450266731101, 23: 0.9631757660075193, 15: 1.8748581102607451, 2: 0.6058779444522936, 8: 0.6854584491551576, 17: 1.6669198358773443, 14: 0.6054717251778557, 6: 0.43145107960433565, 22: 0.5666687973769665, 20: 0.5287380804117249, 16: 0.6819656077609874, 21: 2.6925597594852957, 13: 1.6055760661940772, 5: 2.214602017565466}
2
0.043478260869565216
----------------------------------------
Stage -  test
              precision    recall  f1-score   support

           0       0.55      0.88      0.68        24
           1       0.70      0.29      0.41        24

    accuracy                           0.58        48
   macro avg       0.63      0.58      0.54        48
weighted avg       0.63      0.58      0.54        48

***********

1
0.17073170731707318
{5: 2.370668953279949, 23: 1.0022818400370446, 19: 1.314031120921062, 24: 0.48286795091997836, 17: 1.9164551626055994, 7: 1.152078362137261, 20: 0.580674143386063, 22: 0.6591324490060045, 15: 1.6921522868950394, 11: 2.0199705792581715, 18: 0.7219845547151338, 1: 1.354134585129026, 21: 1.7329242504905258, 6: 0.41933088043142286, 4: 0.5924122852082134, 3: 1.0582217443434123, 10: 0.4807727232586025, 16: 0.6548020005464781, 12: 0.5117949056959092, 13: 1.4112833374130693, 8: 0.6028109744146812, 2: 0.528289319163238, 9: 1.1136604172776665, 14: 0.5993585046102652}
2
0.043478260869565216
----------------------------------------
Stage -  test
              precision    recall  f1-score   support

           0       0.56      0.83      0.67        24
           1       0.67      0.33      0.44        24

    accuracy                           0.58        48
   macro avg       0.61      0.58      0.56        48
weighted avg       0.61      0.58      0.56        48

*********

...

2
0.054945054945054944
{8: 0.5079370336932156, 16: 0.4969294546682732, 14: 0.4603188686967255, 17: 3.2718334680654726, 23: 1.3664716091170632, 12: 0.397151699523049, 9: 3.9817751763444504, 3: 1.11437324518735, 5: 1.505822554143801, 2: 0.3642074791956214, 24: 0.3620625070945533, 1: 1.3906694628102363, 20: 0.43932998263919165, 22: 0.5346955831738558, 4: 0.35170685783520916, 7: 0.9738166112695391, 11: 1.8638595556945035, 19: 0.8863565882325438, 21: 2.055452254905501, 10: 0.30087534570324903, 18: 0.5244181141368952, 15: 3.0364765729722856, 13: 2.44647917951567, 6: 0.2673343436473394}
3
0.043478260869565216
----------------------------------------
Stage -  test
              precision    recall  f1-score   support

           0       0.58      0.92      0.71        24
           1       0.80      0.33      0.47        24

    accuracy                           0.62        48
   macro avg       0.69      0.62      0.59        48
weighted avg       0.69      0.62      0.59        48

********

4
0.054945054945054944
{20: 0.38414475299664413, 23: 1.723632227191148, 4: 0.3075643798030056, 12: 0.2716483064292176, 10: 0.18289377770849546, 13: 10.40492132617277, 11: 3.678155702184692, 7: 2.2104093450195372, 14: 0.2944094900969539, 17: 1.6496246333665823, 6: 0.1404481379413203, 8: 0.2863637175508497, 3: 0.4875888610482503, 22: 0.3458582368116019, 1: 1.2499420933475907, 21: 11.96958893760526, 18: 0.4187663264118568, 24: 0.22383241559795988, 19: 1.3420356843767747, 2: 0.2807594349266022, 16: 0.3929852608712985, 5: 9.195464429514043, 9: 10.854903771629335, 15: 2.527074689063273}
5
0.043478260869565216
----------------------------------------
Stage -  test
              precision    recall  f1-score   support

           0       0.61      0.96      0.74        24
           1       0.90      0.38      0.53        24

    accuracy                           0.67        48
   macro avg       0.75      0.67      0.64        48
weighted avg       0.75      0.67      0.64        48

*******

6
0.06666666666666667
{11: 9.319750402789692, 3: 2.528195179930628, 15: 18.3987122878298, 5: 12.47166828853721, 16: 0.2685412111044814, 22: 0.22059813259845182, 6: 0.03680673853072213, 9: 3.319346648063093, 2: 0.1461937295217555, 10: 0.1720876595949044, 20: 0.13854505194295852, 7: 2.0662210860178574, 8: 0.1431196171470151, 4: 0.18276868893516957, 13: 4.994235697559491, 14: 0.14746420358370674, 21: 3.2351282119635205, 1: 6.6055556670022115, 12: 0.17274396805735362, 18: 0.12462491671340435, 24: 0.08252321307221878, 17: 6.291404857164757, 19: 0.7606296821869907, 23: 0.8040631895560865}
7
0.03225806451612903
----------------------------------------
Stage -  test
              precision    recall  f1-score   support

           0       0.44      0.71      0.54        24
           1       0.22      0.08      0.12        24

    accuracy                           0.40        48
   macro avg       0.33      0.40      0.33        48
weighted avg       0.33      0.40      0.33        48

******

0
1000000
{3: 1.1250036466325553, 10: 0.7107899441098232, 7: 1.2199145296232632, 22: 0.8096798210337282, 9: 2.1208490428157103, 8: 0.8254949272396244, 21: 1.6674130595565217, 24: 0.6579787837969706, 13: 1.0849542505815342, 11: 1.189072759330154, 5: 1.1043853962601549, 19: 1.3962753995561188, 18: 0.8435191848498762, 23: 1.1979119238445168, 16: 0.8573426814987762, 17: 1.4425964187043074, 1: 1.6305965226903996, 12: 0.7350532606895102, 4: 0.7696832369281622, 6: 0.640865620235181, 20: 0.834474126284139, 15: 1.3298471391038102, 2: 0.7328230546672254, 14: 0.7495016111261299}
1
0.11627906976744186
{3: 1.2008284143910375, 10: 0.5052223446476455, 7: 1.4881914595859473, 22: 0.65558141258921, 9: 3.675201856583795, 8: 0.6814418748983526, 21: 2.7802663111796404, 24: 0.4329360799269405, 13: 1.1771257258549386, 11: 1.4138940269810254, 5: 1.2196671034726991, 19: 1.1269995749279489, 18: 0.7115246152097995, 23: 1.434992977288871, 16: 0.7350364735195124, 17: 1.774748793684116, 1: 1.9203526115191096, 12: 0

2
0.09090909090909091
{24: 0.501888564142138, 8: 0.5291685570850747, 13: 2.225698528770789, 11: 1.954812963287871, 10: 0.39090600047357077, 20: 0.3929258326739243, 2: 0.4820434345958395, 16: 0.601901955639917, 3: 1.0604130851760316, 5: 1.3273107675061073, 7: 1.3891631977560965, 22: 0.4713733050830758, 9: 1.4357305835042762, 19: 0.8834545072152844, 6: 0.2570869820927398, 4: 0.515624613219189, 23: 1.2068217341927556, 14: 0.45895201928758567, 1: 1.6954114248338932, 18: 0.6925996907655875, 21: 2.6856549956822313, 17: 3.1214300820490424, 15: 3.1179359151795434, 12: 0.38053530086849774}
3
0.043478260869565216
----------------------------------------
Stage -  test
              precision    recall  f1-score   support

           0       0.59      0.92      0.72        24
           1       0.82      0.38      0.51        24

    accuracy                           0.65        48
   macro avg       0.71      0.65      0.62        48
weighted avg       0.71      0.65      0.62        48

*******

In [192]:
to_dump = {'clf_report':ITERATION_CLF_REPORT, 'file_change':ITERATION_FILE_CHANGE, 'ldc_clfs':LDC_CLFS, 'gmms': trained_GMMs}

In [193]:
with open(WRITE_PATH+EXPERIMENT_TYPE+'.pickle', 'wb') as handle:
    pickle.dump(to_dump, handle, protocol=pickle.HIGHEST_PROTOCOL)